In [1]:
from pypdf import PdfReader
from openai import OpenAI
import json
from pathlib import Path
import csv

API_KEY = 'sk-bedceae2ceba437f944db22706354095'
import os

In [2]:
def make_prompt_dividend_detection():
    return """
You are a financial expert.

Go through this content and check if there is any mention of dividend per share

Your task is to find the dividend the company pays per share.

Your output must be in JSON format with the following keys:

{
  "is_dividend_mentioned": true/false,
  "dividend_amount": "..."
}

Avoid making assumptions beyond the text.
"""

def make_prompt_dividend_calculation():
    return """
You are a financial expert.

Go through this content and assimilate all the information about dividends

Your task is to find the dividend the company pays in all forms.

Your output must be in JSON format with the following keys:

{
  "dividend_amount_paid": "..."
}

Avoid making assumptions beyond the text. Dont put anything extra, just one absolute value, dividend per share. 
So it should not be a large value
Do not put any text like per share or currency name or %. if you cannot find anything put 0
"""

In [3]:
def make_dividend_context(res_dic):
    context_string=''
    for res_dic in results:
        if res_dic['is_dividend_mentioned']==True:
            #print(res_dic)            
            context_string+=res_dic['page_text']+'\n'
    return context_string
            
                

In [4]:
#results

In [5]:
company_codes=['QNBK','QFBQ','QNCD','QNNS','SIIS','ZHCD']

In [6]:

api_client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")
output_csv = Path("dividend_history_per_share.csv")

# Load existing data with proper nested structure
existing_data = {}
if output_csv.exists():
    with open(output_csv, mode='r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            company = row['company_code']
            year = int(row['year'])
            amount = float(row['dividend_amount_paid'])
            
            if company not in existing_data:
                existing_data[company] = {}
            existing_data[company][year] = amount


for company_code in company_codes:


    
    # read through all annual reports in the folder
    report_files_location=f'../extra/{company_code}_annual_reports/'
    for file_name in os.listdir(report_files_location):
        if '.DS' in file_name:
            continue    
        year = int(file_name.split('.pdf')[0])
        
        if company_code in existing_data and year in existing_data[company_code]:
            print(f'Skipping {company_code}, {year} - already in CSV')
            continue
                
        print(file_name,year)
    
        try:
            reader = PdfReader(f"../extra/{company_code}_annual_reports/{file_name}")
        except:
            print('issues reading')
            continue
        print(len(reader.pages))
        system_prompt=make_prompt_dividend_detection()
        results=[]
        for page_num in range(len(reader.pages)):        
            page = reader.pages[page_num]
            #print(page.extract_text())
            text=page.extract_text()
            if len(text)<50:
                continue
            if 'dividen' not in text.lower():
                continue
            print(f'will analyze page {page_num}')
            response = api_client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": text}
                ],
                response_format={'type': 'json_object'}
            )    
            #print(response.choices[0].message.content)
            dic_res = json.loads(response.choices[0].message.content)
            dic_res['page_num']=page_num
            dic_res['page_text']=text
            results.append(dic_res)
            print('*'*10)
    
        context_string=make_dividend_context(dic_res)
        system_prompt = make_prompt_dividend_calculation()
        response = api_client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": context_string}
            ],
            response_format={'type': 'json_object'}
        )    
        print(response.choices[0].message.content)
        dic_res = json.loads(response.choices[0].message.content)    
        dividend_amount_paid=dic_res['dividend_amount_paid']
        if ',' in dividend_amount_paid:
            dividend_amount_paid=dividend_amount_paid.replace(',','')
        try:
            print(float(dividend_amount_paid))         
            dividend_amount_paid=float(dividend_amount_paid)
        except:
            print(f'something fishy in {dividend_amount_paid}')
            continue

        # Append to CSV after each successful year
        with open(output_csv, mode='a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=['company_code', 'year', 'dividend_amount_paid'])
            if f.tell() == 0:  # Write header if file is empty
                writer.writeheader()
            writer.writerow({
                'company_code': company_code,
                'year': year,
                'dividend_amount_paid': dividend_amount_paid
            })
            
        print(f"Saved {year}: {dividend_amount_paid}")
    

Skipping QNBK, 2015 - already in CSV
Skipping QNBK, 2014 - already in CSV
Skipping QNBK, 2016 - already in CSV
Skipping QNBK, 2017 - already in CSV
Skipping QNBK, 2013 - already in CSV
Skipping QNBK, 2012 - already in CSV
Skipping QNBK, 2010 - already in CSV
Skipping QNBK, 2011 - already in CSV
Skipping QNBK, 2008 - already in CSV
2020.pdf 2020
84
will analyze page 4
**********
will analyze page 12
**********
will analyze page 54
**********
will analyze page 55
**********
will analyze page 59
**********
will analyze page 60
**********
will analyze page 61
**********
will analyze page 69
**********
will analyze page 73
**********
will analyze page 76
**********
will analyze page 77
**********
will analyze page 78
**********
{
  "dividend_amount_paid": "0.45"
}
0.45
Saved 2020: 0.45
2021.pdf 2021
85
will analyze page 4
**********
will analyze page 54
**********
will analyze page 55
**********
will analyze page 59
**********
will analyze page 61
**********
will analyze page 70
**********


TypeError: argument of type 'int' is not iterable

In [ ]:

        
# api_client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")
# years=[i for i in range(2015,2025)]

# for year in years:
#     # Skip if already processed
#     if year in existing_data:
#         print(f"Skipping {year} - already in CSV")
#         continue    
#     print(year)
#     file_name=f'04-annual-financial-english-{year}.pdf'
#     try:
#         reader = PdfReader(f"ZHCD.QA/{file_name}")
#     except:
#         print('issues reading')
#         continue
#     print(len(reader.pages))

    
#     system_prompt=make_prompt_dividend_detection()
    
#     results=[]
#     for page_num in range(len(reader.pages)):
#         print(page_num)
#         page = reader.pages[page_num]
#         #print(page.extract_text())
#         text=page.extract_text()
#         if len(text)<50:
#             continue
#         if 'divi' not in text.lower():
#             continue
        
#         response = api_client.chat.completions.create(
#             model="deepseek-chat",
#             messages=[
#                 {"role": "system", "content": system_prompt},
#                 {"role": "user", "content": text}
#             ],
#             response_format={'type': 'json_object'}
#         )    
#         #print(response.choices[0].message.content)
#         dic_res = json.loads(response.choices[0].message.content)
#         dic_res['page_num']=page_num
#         dic_res['page_text']=text
#         results.append(dic_res)
#         print('*'*10)

#     context_string=make_dividend_context(dic_res)
#     system_prompt = make_prompt_dividend_calculation()
#     response = api_client.chat.completions.create(
#         model="deepseek-chat",
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": context_string}
#         ],
#         response_format={'type': 'json_object'}
#     )    
#     print(response.choices[0].message.content)
#     dic_res = json.loads(response.choices[0].message.content)    
#     dividend_amount_paid=dic_res['dividend_amount_paid']
#     if ',' in dividend_amount_paid:
#         dividend_amount_paid=dividend_amount_paid.replace(',','')
#     print(float(dividend_amount_paid))      
#     # Append to CSV after each successful year
#     with open(output_csv, mode='a', newline='') as f:
#         writer = csv.DictWriter(f, fieldnames=['year', 'dividend_amount_paid'])
#         if f.tell() == 0:  # Write header if file is empty
#             writer.writeheader()
#         writer.writerow({
#             'year': year,
#             'dividend_amount_paid': dividend_amount_paid
#         })
        
#     print(f"Saved {year}: {dividend_amount_paid}")


In [ ]:
# results

In [ ]:
# context_string

In [ ]:
print(dic_res['dividend_amount_paid'])

In [ ]:
page = reader.pages[9]
        
text=page.extract_text()

In [ ]:
text

In [ ]:
len(context_string)

In [ ]:
context_string=make_dividend_context(dic_res)

In [ ]:
context_string

In [ ]:
system_prompt = make_prompt_dividend_calculation()
response = api_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": context_string}
    ],
    response_format={'type': 'json_object'}
)    
print(response.choices[0].message.content)